In [ ]:
!pip install faker

In [ ]:
# 1. Import required components
from website import create_app, db  # Import the create_app function from your website module
from website.models import User, Device, DeviceData  # Adjust this import to match where your models are located
from faker import Faker
import random
from werkzeug.security import generate_password_hash
from datetime import datetime, timedelta

# 2. Initialize the app and push the context
app = create_app()
app.app_context().push()

# Initialize Faker
faker = Faker()

# Ensure the database schema is created (in case it hasn't been)
db.create_all()

In [ ]:
def create_fake_user(num_users=1):
    users = []
    for _ in range(num_users):
        # Generate a fake user
        email = faker.email()
        password = generate_password_hash(faker.password(), method='pbkdf2:sha256')
        first_name = faker.first_name()

        user = User(email=email, password=password, first_name=first_name)
        db.session.add(user)
        users.append(user)
    
    db.session.commit()
    return users

# Create 5 fake users
fake_users = create_fake_user(5)


In [ ]:
def add_device_to_user(user, num_devices=1):
    devices = []
    for _ in range(num_devices):
        # Generate a fake device for the user
        device_name = faker.word() + " Device"
        device_type = random.choice(['Sensor', 'Monitor', 'Tracker'])
        serial_number = faker.unique.uuid4()

        device = Device(name=device_name, type=device_type, serial_number=serial_number, user_id=user.id)
        db.session.add(device)
        devices.append(device)
    
    db.session.commit()
    return devices

# Add 2 devices to each user
for user in fake_users:
    add_device_to_user(user, num_devices=2)


In [ ]:
def add_device_data(device, num_entries=10):
    for _ in range(num_entries):
        timestamp = faker.date_time_between(start_date="-30d", end_date="now")
        value1 = random.uniform(0.0, 100.0)  # Random float between 0 and 100
        value2 = random.uniform(0.0, 100.0)

        device_data = DeviceData(device_id=device.id, timestamp=timestamp, value1=value1, value2=value2)
        db.session.add(device_data)
    
    db.session.commit()

# Add fake data to all devices
all_devices = Device.query.all()
for device in all_devices:
    add_device_data(device, num_entries=10)
